In [23]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
from tvDatafeed import TvDatafeed, Interval

In [24]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [36]:
# queries
query_today = """SELECT s.symbol as symbol, s.exchange as exchange
                 FROM securities s INNER JOIN positions p ON s.symbol = p.symbol
                 WHERE p.quote_last_quote_at < CURRENT_DATE AND s.category='stock'"""
query_last_hour = """SELECT s.symbol, s.exchange, DATE_TRUNC('hour', now()) - quote_last_quote_at AS datediff
                     FROM securities s INNER JOIN positions p ON s.symbol = p.symbol
                     WHERE quote_last_quote_at < DATE_TRUNC('hour', now())
"""
securities = None
with engine.begin() as conn:
  #  SELECT symbol, exchange FROM securities WHERE category='stock'
  securities = run_sql(conn, query_last_hour).mappings().all()
securities

[{'symbol': 'DBK', 'exchange': 'XETR', 'datediff': datetime.timedelta(seconds=3600)},
 {'symbol': 'INMD', 'exchange': 'NASDAQ', 'datediff': datetime.timedelta(seconds=14400)},
 {'symbol': 'CI', 'exchange': 'NYSE', 'datediff': datetime.timedelta(seconds=3600)},
 {'symbol': 'ESEA', 'exchange': 'NASDAQ', 'datediff': datetime.timedelta(seconds=61200)},
 {'symbol': 'GCT', 'exchange': 'NASDAQ', 'datediff': datetime.timedelta(seconds=14400)},
 {'symbol': 'NKTX', 'exchange': 'NASDAQ', 'datediff': datetime.timedelta(seconds=3600)},
 {'symbol': 'GOOG', 'exchange': 'NASDAQ', 'datediff': datetime.timedelta(seconds=10800)},
 {'symbol': 'LQQ', 'exchange': 'EURONEXT', 'datediff': datetime.timedelta(seconds=3600)},
 {'symbol': 'AAOI', 'exchange': 'NASDAQ', 'datediff': datetime.timedelta(seconds=3600)},
 {'symbol': 'HEI', 'exchange': 'NYSE', 'datediff': datetime.timedelta(seconds=57600)},
 {'symbol': 'JPM', 'exchange': 'NYSE', 'datediff': datetime.timedelta(seconds=10800)},
 {'symbol': 'CSCO', 'exchang

In [37]:
from helpers_db import insert_on_conflict_nothing_ticks, truncate_table

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, bars=100):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=Interval.in_1_hour,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

# with engine.begin() as conn:
#   truncate_table(conn, 'ticks')

for security in securities:
  symbol    = security['symbol']
  exchange  = security['exchange']
  print(f"Processing {exchange}:{symbol}")
  try:
    df = fetch_ticks(symbol, exchange, bars=7*24)
    print(f"  count: {df.shape[0]}")
    df = normalize_dataframe(df)
    df.to_sql('ticks', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
  except Exception as e:
    print(f"Exception {exchange}:{symbol}")
    print(str(e))


Processing XETR:DBK
  count: 168
Processing NASDAQ:INMD
  count: 168
Processing NYSE:CI
  count: 168
Processing NASDAQ:ESEA
  count: 168
Processing NASDAQ:GCT
  count: 168
Processing NASDAQ:NKTX
  count: 168
Processing NASDAQ:GOOG
  count: 168
Processing EURONEXT:LQQ
  count: 168
Processing NASDAQ:AAOI
  count: 168
Processing NYSE:HEI
  count: 168
Processing NYSE:JPM
  count: 168
Processing NASDAQ:CSCO
  count: 168
Processing NYSE:CLS
  count: 168
Processing NASDAQ:CRDF
  count: 168
Processing NASDAQ:AMZN
  count: 168
Processing NASDAQ:ON
  count: 168
Processing NASDAQ:SEDG
  count: 168
Processing NASDAQ:SURG
  count: 168
Processing NASDAQ:WDAY
  count: 168
